In [33]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [34]:
import numpy as np
import pandas as pd

df = pd.read_csv('/content/drive/MyDrive/logs_dataset.csv')

print(df.head())
print(df.shape)
print(df.info())
print(df.describe())
print(df.isnull().sum())


# import nltk
import regex


                    @timestamp                   _id  ip_address
0  July 8th 2019, 14:43:03.000  XswJ0msBoTGddM7vxMDB  10.1.1.285
1  July 8th 2019, 14:43:01.000  dKQJ0msB7mP0GwVzvJjz  10.1.2.389
2  July 8th 2019, 14:42:59.000  CcwJ0msBoTGddM7vtb8y  10.1.1.415
3  July 8th 2019, 14:42:57.000  bKQJ0msB7mP0GwVzrZdT   10.1.1.79
4  July 8th 2019, 14:42:55.000  L6QJ0msB7mP0GwVzpZeI   10.1.1.60
(721547, 3)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 721547 entries, 0 to 721546
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   @timestamp  721547 non-null  object
 1   _id         721547 non-null  object
 2   ip_address  721547 non-null  object
dtypes: object(3)
memory usage: 16.5+ MB
None
                          @timestamp     _id ip_address
count                         721547  721547     721547
unique                        706029  712431        386
top     June 11th 2019, 15:23:06.001  #NAME?  10.1.2.63
freq         

In [35]:
df['@timestamp'] = (
    df['@timestamp']
    .str.replace(r'(\d+)(st|nd|rd|th)', r'\1', regex=True)
)
df['@timestamp'] = pd.to_datetime(
    df['@timestamp'],
    format='%B %d %Y, %H:%M:%S.%f'
)

In [36]:
df.sort_values(['ip_address','@timestamp'], inplace=True)

In [37]:
df['shift_time'] = df.groupby(['ip_address'])['@timestamp'].shift(1)
print(df.head)

<bound method NDFrame.head of                 @timestamp                   _id ip_address  \
721473 2019-06-09 00:06:09  DBuOOWsB7mP0GwVzhZ9U   10.1.1.1   
720483 2019-06-09 01:28:39  bB7aOWsB7mP0GwVzDY5G   10.1.1.1   
719233 2019-06-09 03:12:49  R0w5OmsBoTGddM7vayZT   10.1.1.1   
719222 2019-06-09 03:13:45  U0w6OmsBoTGddM7vRi8R   10.1.1.1   
718875 2019-06-09 03:42:39  z01UOmsBoTGddM7vuzyC   10.1.1.1   
...                    ...                   ...        ...   
3225   2019-07-08 11:49:00  3cBq0WsBoTGddM7va5TJ  10.1.2.99   
2422   2019-07-08 12:22:29  QMKJ0WsBoTGddM7vE9N1  10.1.2.99   
1704   2019-07-08 12:52:25  9pyk0WsB7mP0GwVze7sV  10.1.2.99   
1138   2019-07-08 13:15:59  B8a60WsBoTGddM7vDnqQ  10.1.2.99   
368    2019-07-08 13:47:19  M8jW0WsBoTGddM7vvplE  10.1.2.99   

                shift_time  
721473                 NaT  
720483 2019-06-09 00:06:09  
719233 2019-06-09 01:28:39  
719222 2019-06-09 03:12:49  
718875 2019-06-09 03:13:45  
...                    ...  
3225   201

In [38]:
df['time_diff'] = (df['@timestamp']-df['shift_time']).dt.seconds//60

In [40]:
df['date'] = df['@timestamp'].dt.date

In [41]:
df['weekday'] = df['@timestamp'].dt.weekday

In [42]:
df['hour'] = df['@timestamp'].dt.hour

In [43]:
df['is_weekend'] = ((df['weekday'] == 5) | (df['weekday'] ==6)).astype(int)

In [44]:
df['hour_bucket'] = df['hour']//4

In [45]:
ip_addr = 'ip_address'

In [49]:
ip_counts = df.groupby(ip_addr)['@timestamp'].count().reset_index()

In [50]:
ip_counts.head()

,ip_address,@timestamp
0,10.1.1.1,1446
1,10.1.1.100,2860
2,10.1.1.101,1465
3,10.1.1.106,1408
4,10.1.1.109,1459


In [39]:
import re

# Define a regex pattern to extract the full timestamp string
timestamp_pattern = re.compile(r'([A-Za-z]+ \d{1,2}(?:st|nd|rd|th)?, \d{4}, \d{2}:\d{2}:\d{2}\.\d{3})')

# Apply the regex to the '@timestamp' column
df['extracted_timestamp_str'] = df['@timestamp'].apply(lambda x: timestamp_pattern.search(x).group(1) if timestamp_pattern.search(x) else None)

# Convert the extracted string to datetime objects
df['timestamp_datetime'] = pd.to_datetime(df['extracted_timestamp_str'], format='%B %dth %Y, %H:%M:%S.%f', errors='coerce')

# Handle variations for 'st', 'nd', 'rd'
def parse_date_with_ordinal(date_str):
    if date_str is None:
        # return None
    # Remove 'st', 'nd', 'rd', 'th' for parsing
    date_str = re.sub(r'(\d+)(st|nd|rd|th)', r'\1', date_str)
    try:
        return pd.to_datetime(date_str, format='%B %d %Y, %H:%M:%S.%f')
    except ValueError:
        return None

df['timestamp_datetime'] = df['@timestamp'].apply(parse_date_with_ordinal)

print("First 5 rows with extracted timestamps:")
print(df[['@timestamp', 'timestamp_datetime']].head())
print("\nInfo on the new datetime column:")
print(df['timestamp_datetime'].info())
print("\nNumber of nulls in the new datetime column:")
print(df['timestamp_datetime'].isnull().sum())

TypeError: expected string or bytes-like object, got 'Timestamp'